In [32]:
import pandas as pd
import numpy as np
import scipy
from tableone import TableOne
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Epitope Database

In [33]:
path_cohort = '/Users/Danial/UMCUtrecht/ProcessedData/procare_1_cohort/procare_1_cohort.xlsx'
df_cohort = pd.read_excel(path_cohort, skiprows=1)
# path_cohort = '~/UMCUtrecht/KaplanMeier/DESAsurvival_original.csv'
# df_cohort = pd.read_csv(path_cohort, sep=';', decimal=',')
path_ajt = '/Users/Danial/UMCUtrecht/KaplanMeier/Kambruva_KM_Suplimentary_Data.xlsx'
df_ajt = pd.read_excel(path_ajt)[['TransplantID', 'DSA']]

In [34]:
df = df_ajt.merge(df_cohort, on='TransplantID')
df.shape

(4724, 132)

# Missing CIPs
 The data set procare_1_cohort does not consider the imputed CIP hours. While the dataset DESAsurvival_original does consider the missing values

In [122]:
df.columns[30:50]
# df['InitialImmunosupp_NOTR'].values

Index(['cPRA_ETgrps', 'HighPRA_NOTR', 'hPRA_ETgrps', 'Dialysis days (renine)',
       'Last dialysis technique (renine)', 'DialysisYears',
       'Initial Primary Disease', 'Initial Primary Disease (ET)',
       'Initial Primary Disease (renine)', 'DialysisYN', 'PrimaryDisease',
       'MembranousNephropathy', 'CysticAndCongenitalDiseases',
       'DiabetesType_I', 'DiabetesType_II', 'Glomerulonephritis',
       'Pyelo_Interstitial_Nephritis', 'OtherRenalDisease',
       'VascularRenalDisease', 'UnknownRenalDisease'],
      dtype='object')

In [125]:
df['DialysisYN'].value_counts()

Yes_haemo    2441
Yes_peri     1712
No            542
Unknown        29
Name: DialysisYN, dtype: int64

In [64]:
df[['CIPHour']].isna().sum()

CIPHour    227
dtype: int64

In [36]:
# 'Steroids_T0', 'MMF_T0', 'Aza_T0', 'Sirolimus_T0', 'Cyclosporin_T0', 'Tacrolimus_T0', 'Unknown_T0', 'Others_T0', CalcineurinInhibitor_T0, AntiProliferativeAgents_T0, ATG_T0, OKT3_T0, ATG_OKT3
# df['Steroids_T0'].eq('Yes').sum()
# df['Cyclosporin_T0'].eq('Yes').sum() + df['Tacrolimus_T0'].eq('Yes').sum()
df['MMF_T0'].eq('Yes').sum()
# df['ALG_ATG_OKT3_AntiPanTMoAb'].eq('Yes').sum()
# print(df['MMF_T0'].eq('Yes').sum() + df['Aza_T0'].eq('Yes').sum())
a = df[['MMF_T0', 'Aza_T0']].apply(lambda x: 'Yes'  if ((x[0] == 'Yes') | (x[1] == 'Yes')) else 'No', axis=1)
a.value_counts()

Yes    3605
No     1119
dtype: int64

###  Table 1

In [73]:
# selected variables
cols = [
    'DSA', 
    'RecipientAge_NOTR', 'RecipientSex_NOTR', 'DialysisYears',
    'DonorAge_NOTR', 'DonorSex_NOTR', 
    'TypeCadaveric_NOTR', 
    'CIPHour_DBD', 'CIPHour_DCD',
    'CIPHour',
    'Retransplant', 
    'CurrentPRA_NOTR',
    'HighPRA_NOTR',
    'ALG_ATG_OKT3_AntiPanTMoAb',
    'IL2rMoAb_T0',
    'NumMismatch_ABDR_broad',
    'Steroids_T0', 'MMF_T0', 'Aza_T0', 'Sirolimus_T0', 'Cyclosporin_T0', 'Tacrolimus_T0', 'Unknown_T0', 'Others_T0',
    'DialysisYN',
    'TypeOfDonor_NOTR'
]
df_t1 = df[cols]
df_t1['TypeCadaveric_NOTR'].fillna('Living', inplace=True)

df_t1['CIPDeceadDonor'] = (df_t1['CIPHour_DBD'] + df_t1['CIPHour_DCD']).replace(0, np.nan)
for g, v in df.groupby('TypeOfDonor_NOTR')['CIPHour']:
    if g == 'Living':
        df_t1['CIPLivingDonor'] = v
df_t1.drop(['CIPHour_DBD', 'CIPHour_DCD', 'CIPHour'], axis =1, inplace=True)
df_t1['MMF/azathioprine'] = df[['MMF_T0', 'Aza_T0']].apply(lambda x: 'Yes'  if ((x[0] == 'Yes') | (x[1] == 'Yes')) else 'No', axis=1)
df_t1['Cyclosporin/Tacrolimus'] = df[['Cyclosporin_T0', 'Tacrolimus_T0']].apply(lambda x: 'Yes'  if ((x[0] == 'Yes') | (x[1] == 'Yes')) else 'No', axis=1)
df_t1.drop(['Cyclosporin_T0', 'Tacrolimus_T0', 'MMF_T0', 'Aza_T0'], axis =1, inplace=True)

/var/folders/dz/h73npvns6dscvynm6d4050wc0000gn/T/ipykernel_2147/904228272.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t1['TypeCadaveric_NOTR'].fillna('Living', inplace=True)
/var/folders/dz/h73npvns6dscvynm6d4050wc0000gn/T/ipykernel_2147/904228272.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t1['CIPDeceadDonor'] = (df_t1['CIPHour_DBD'] + df_t1['CIPHour_DCD']).replace(0, np.nan)
/var/folders/dz/h73npvns6dscvynm6d4050wc0000gn/T/ipykernel_2147/904228272.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [38]:
df_t1.to_csv('/Users/Danial/UMCUtrecht/ProcessedData/tableone.csv')

In [39]:
'MMF/azathioprine' in df_t1.columns
'Cyclosporin/Tacrolimus' in df_t1.columns

True

In [40]:
# columns containing categorical variables
categorical = ['RecipientSex_NOTR', 'DonorSex_NOTR', 
'IL2rMoAb_T0', 'ALG_ATG_OKT3_AntiPanTMoAb',
'Retransplant', 'TypeCadaveric_NOTR', 'DialysisYN',
'Steroids_T0', 'Sirolimus_T0', 'MMF/azathioprine', 'Cyclosporin/Tacrolimus', 'Unknown_T0', 'Others_T0',
]

# optionally, a categorical variable for stratification
groupby = ['DSA']

# rename the death column
labels={
    'DonorAge_NOTR': 'Donor Age',
    'DonorSex_NOTR': 'Donor Sex',
    'RecipientAge_NOTR': 'Recipiet Age',
    'RecipientSex_NOTR': 'Recipiet Sex',
    # 'DialysisYears': 'Time on dialysis, y',
    # 'TypeCadaveric_NOTR': 'Type of Donor',
    # 'CIPDeceadDonor': ' Cold Ischemia Time Deceased Donors', 
    # 'CIPLivingDonor': ' Cold Ischemia Time Living Donors', 
    # 'CurrentPRA_NOTR': 'PRA at time of transplant, %',
    # 'HighPRA_NOTR': 'Highest PRA, %',
    # 'NumMismatch_ABDR_broad': 'HLA-A/B/DR broad mismatches, mean ± SD', 
    # 'Dialysis': 'DialysisYN',
}

continuous = {
    'DonorAge_NOTR', 'RecipientAge_NOTR', 'CIPDeceadDonor', 'DialysisYears',
    'CIPLivingDonor', 'CurrentPRA_NOTR', 'HighPRA_NOTR', 'NumMismatch_ABDR_broad',
    }
# htest = {var:scipy.stats.mannwhitneyu for var in continuous}

htest = {'RecipientAge_NOTR': scipy.stats.mannwhitneyu}

# link to documentation https://github.com/tompollard/tableone/blob/master/tableone/tableone.py

table1 = TableOne(
    df_t1, 
    groupby=groupby,
    missing=True,
    categorical=categorical,
    pval=True,
    rename=labels,
    htest_name=True,
    # htest=htest,
)
table1

Grouped by DSA                                                                                          
                                                          Missing      Overall            0            1 P-Value                                       Test
n                                                                         4724         4157          567                                                   
Recipiet Age, mean (SD)                                         0  45.5 (14.4)  45.6 (14.4)  44.2 (13.9)   0.022                          Two Sample T-test
Recipiet Sex, n (%)               Female                        0  1894 (40.1)  1561 (37.6)   333 (58.7)  <0.001                                Chi-squared
                                  Male                             2830 (59.9)  2596 (62.4)   234 (41.3)                                                   
DialysisYears, mean (SD)                                       28    2.7 (2.5)    2.7 (2.4)    3.4 (3.0)  <0.001                          Two Sample T-test
Donor Age, mean (SD)                                            0  44.4 (15.0)  44.4 (14.9)  44.0 (15.8)   0.539                          Two Sample T-test
Donor Sex, n (%)                  Female                        0  2386 (50.5)  2128 (51.2)   258 (45.5)   0.011                                Chi-squared
                                  Male                             2338 (49.5)  2029 (48.8)   309 (54.5)                                                   
TypeCadaveric_NOTR, n (%)         Heartbeating                  0  2427 (51.4)  2076 (49.9)   351 (61.9)  <0.001                                Chi-squared
                                  Living                           1487 (31.5)  1350 (32.5)   137 (24.2)                                                   
                                  Non-heartbeating                  810 (17.1)   731 (17.6)    79 (13.9)                                                   
Retransplant, n (%)               No                            0  4001 (84.7)  3704 (89.1)   297 (52.4)  <0.001                                Chi-squared
                                  Yes                               723 (15.3)   453 (10.9)   270 (47.6)                                                   
CurrentPRA_NOTR, mean (SD)                                     42   6.0 (17.5)   3.5 (12.9)  24.4 (30.7)  <0.001                          Two Sample T-test
HighPRA_NOTR, mean (SD)                                        42  13.8 (25.8)   9.8 (21.0)  43.6 (36.4)  <0.001                          Two Sample T-test
ALG_ATG_OKT3_AntiPanTMoAb, n (%)  No                            0  4540 (96.1)  4012 (96.5)   528 (93.1)  <0.001                                Chi-squared
                                  Yes                                184 (3.9)    145 (3.5)     39 (6.9)                                                   
IL2rMoAb_T0, n (%)                No                            0  3702 (78.4)  3244 (78.0)   458 (80.8)   0.137                                Chi-squared
                                  Yes                              1022 (21.6)   913 (22.0)   109 (19.2)                                                   
NumMismatch_ABDR_broad, mean (SD)                              39    2.4 (1.5)    2.3 (1.5)    2.4 (1.3)   0.092                          Two Sample T-test
Steroids_T0, n (%)                No                            0    108 (2.3)     88 (2.1)     20 (3.5)   0.035                                Chi-squared
                                  Yes                              4616 (97.7)  4069 (97.9)   547 (96.5)                                                   
Sirolimus_T0, n (%)               No                            0  4438 (93.9)  3897 (93.7)   541 (95.4)   0.118                                Chi-squared
                                  Yes                                286 (6.1)    260 (6.3)     26 (4.6)                                               

In [97]:
df_t1['DialysisYN'].value_counts()

Yes_haemo    2441
Yes_peri     1712
No            542
Unknown        29
Name: DialysisYN, dtype: int64

In [110]:
from scipy.stats import mannwhitneyu, chi2_contingency, fisher_exact
def mannwhitneyu_test(data, column, **kwargs):
    dsa_group = data[data.DSA == 1][column]
    print(round(dsa_group.mean(), 2), round(dsa_group.std(), 2))
    no_dsa_group = data[data.DSA == 0][column]
    print(round(no_dsa_group.mean(), 2), round(no_dsa_group.std(), 2))
    print(mannwhitneyu(dsa_group, no_dsa_group, **kwargs))

def chi2_test(data, column, fisher_test=False, **kwargs):
    # https://pythonfordatascienceorg.wordpress.com/chi-square-python/
    group_by = data['DSA']
    cat_var = data[column]
    crosstab = pd.crosstab(group_by, cat_var)
    if fisher_test:
        print(fisher_exact(crosstab))
    else:
        print(chi2_contingency(crosstab, **kwargs))
    return crosstab

# Redoing the tests & p_values
Instead of two-sided t-test we are using Mann-Whitny test for continuous variables. 

In [53]:
mannwhitneyu_test(df, 'DonorAge_NOTR')

43.98 15.8
44.41 14.9
MannwhitneyuResult(statistic=1178106.0, pvalue=0.9894430286898469)


In [51]:
mannwhitneyu_test(df, 'RecipientAge_NOTR')

44.19 13.92
45.62 14.42
MannwhitneyuResult(statistic=1103821.5, pvalue=0.014200179801494727)


In [61]:
mannwhitneyu_test(df, 'DialysisYears', nan_policy='omit')

3.36 2.95
2.65 2.39
MannwhitneyuResult(statistic=1315863.5, pvalue=1.568274749095552e-07)


In [54]:
mannwhitneyu_test(df_t1, 'NumMismatch_ABDR_broad', nan_policy='omit')

2.44 1.3
2.34 1.51
MannwhitneyuResult(statistic=1208047.0, pvalue=0.14995837320558258)


In [52]:
mannwhitneyu_test(df_t1, 'CIPDeceadDonor', nan_policy='omit')

22.82 6.8
21.69 7.26
MannwhitneyuResult(statistic=633865.0, pvalue=0.0010082359182155983)


In [66]:
mannwhitneyu_test(df_t1, 'CIPLivingDonor', nan_policy='omit')

2.48 0.97
2.5 1.6
MannwhitneyuResult(statistic=76324.0, pvalue=0.7607679966908673)


In [76]:
df_t1[df_t1.TypeOfDonor_NOTR == 'Living']['CIPLivingDonor'].isna().sum()

152

In [70]:
df_t1.columns#['CIPLivingDonor']

Index(['DSA', 'RecipientAge_NOTR', 'RecipientSex_NOTR', 'DialysisYears',
       'DonorAge_NOTR', 'DonorSex_NOTR', 'TypeCadaveric_NOTR', 'Retransplant',
       'CurrentPRA_NOTR', 'HighPRA_NOTR', 'ALG_ATG_OKT3_AntiPanTMoAb',
       'IL2rMoAb_T0', 'NumMismatch_ABDR_broad', 'Steroids_T0', 'Sirolimus_T0',
       'Unknown_T0', 'Others_T0', 'DialysisYN', 'CIPDeceadDonor',
       'CIPLivingDonor', 'MMF/azathioprine', 'Cyclosporin/Tacrolimus'],
      dtype='object')

In [114]:
chi2_test(df_t1, 'Steroids_T0', correction=False)

(4.443076564287405, 0.03504304391481233, 1, array([[  95.03725656, 4061.96274344],
       [  12.96274344,  554.03725656]]))


Steroids_T0,No,Yes
DSA,,
0,88,4069
1,20,547


In [113]:
chi2_test(df_t1, 'Sirolimus_T0', correction=False)

(2.4435319756451443, 0.11801024314883177, 1, array([[3905.32726503,  251.67273497],
       [ 532.67273497,   34.32726503]]))


Sirolimus_T0,No,Yes
DSA,,
0,3897,260
1,541,26


In [119]:
chi2_test(df_t1, 'MMF/azathioprine',  correction=False)

(0.9606908217486836, 0.327012887107603, 1, array([[ 984.69157494, 3172.30842506],
       [ 134.30842506,  432.69157494]]))


MMF/azathioprine,No,Yes
DSA,,
0,994,3163
1,125,442


In [112]:
chi2_test(df_t1, 'Cyclosporin/Tacrolimus', correction=False)

(3.3456285324330697, 0.06738373959746985, 1, array([[ 255.19263336, 3901.80736664],
       [  34.80736664,  532.19263336]]))


Cyclosporin/Tacrolimus,No,Yes
DSA,,
0,265,3892
1,25,542


In [115]:
chi2_test(df_t1, 'Unknown_T0',  fisher_test=True)

(1.5740881458966565, 0.4478991161205596)


Unknown_T0,No,Yes
DSA,,
0,4143,14
1,564,3


In [116]:
chi2_test(df_t1, 'Unknown_T0',  fisher_test=False, correction=False)

(0.5146655796026379, 0.4731261074052232, 1, array([[4.14204043e+03, 1.49595682e+01],
       [5.64959568e+02, 2.04043184e+00]]))


Unknown_T0,No,Yes
DSA,,
0,4143,14
1,564,3


In [42]:
chi2_test(df_t1, 'Others_T0',  correction=False)

8.760796191255936 0.0030777465838544315 1 [[3629.01524132  527.98475868]
 [ 494.98475868   72.01524132]]


Others_T0,No,Yes
DSA,,
0,3607,550
1,517,50


In [45]:
chi2_test(df_t1, 'DialysisYN',  correction=True)

16.325135118182292 0.0009725524073678226 3 [[ 476.94623201   25.51926334 2148.01799323 1506.51651143]
 [  65.05376799    3.48073666  292.98200677  205.48348857]]


DialysisYN,No,Unknown,Yes_haemo,Yes_peri
DSA,,,,
0,495,22,2114,1526
1,47,7,327,186
